In [4]:
def get_precision(y_pred, y_true):
    ## YOUR CODE HERE...
    n = len(y_pred)
    fps,tps = 0,0
    for i in range(n):
        if y_pred[i] > y_true[i]:
            fps+=1
            continue;
#         if y_pred[i] < y_true[i]:
#             fns+=1
#             continue
#         if (y_pred[i] + y_true[i]) == 0:
#             tns+=1
#             continue
        if (y_pred[i] + y_true[i]) > 1:
            tps+=1
            continue
    precision = tps/(tps + fps)
    return precision

def get_recall(y_pred, y_true):
    ## YOUR CODE HERE...
    n = len(y_pred)
    tps, fns = 0,0,0,0
    for i in range(n):
#         if y_pred[i] > y_true[i]:
#             fps+=1
#             continue;
        if y_pred[i] < y_true[i]:
            fns+=1
            continue
#         if (y_pred[i] + y_true[i]) == 0:
#             tns+=1
#             continue
        if (y_pred[i] + y_true[i]) > 1:
            tps+=1
            continue
    recall = tps/(fns + tps)
    return recall

def get_fscore(y_pred, y_true):
    ## YOUR CODE HERE...
    recall = get_recall(y_pred, y_true)
    precision = get_precision(y_pred, y_true)
    fscore = 2/(1/precision + 1/recall)
    return fscore

In [7]:
y_pred = [1,0,1,0,1,0,1,0]
y_true = [0,1,0,0,1,1,1,1]

In [8]:
print(get_recall(y_pred,y_true))
print(get_precision(y_pred,y_true))
print(get_fscore(y_pred,y_true))

0.4
0.5
0.4444444444444444


In [1]:
def load_file(data_file):
    words = []
    labels = []
    with open(data_file,'rt',encoding="utf8") as f:
        i = 0
        for line in f:
            if i > 0:
                line_split = line[:-1].split("\t")
                words.append(line_split[0].lower())
                labels.append(int(line_split[1]))
            i += 1
    return words,labels

In [55]:
import numpy as np
import gzip
from collections import defaultdict
from statistics import median

In [3]:
def all_complex(datafile):
    words, labels = load_file(datafile)
    #preds = np.ones_like(labels)
    preds = [1]*len(labels)
    precision = get_precision(preds,labels)
    recall = get_recall(preds, labels)
    fscore = get_fscore(preds, labels)
    performance = [precision,recall,fscore]
    return performance
            

In [22]:
print(all_complex("data/complex_words_development.txt"))

[0.418, 1.0, 0.5895627644569816]


In [18]:
# function for labeling word length 
def mark(words,threshold):
    preds = []
    for word in words:
        if len(word) >= threshold:
            preds.append(1)
        else:
            preds.append(0)
    return preds  

In [43]:
def word_length_threshold(training_file, development_file):
    # t stands for training_file, d stands for development_file
    w_t, l_t = load_file(training_file)
    w_d, l_d = load_file(development_file)
    # find the max fscore during the iteration
    fscore_max_t = 0
#     fscore_max_d = 0
    precision_t_max = 0
#     precision_d_max = 0
    recall_t_max = 0
#     recall_d_max = 0
    threshold_t = 0
#     threshold_d = 0
    for i in range(10):
        pred_t = mark(w_t,i)
        precision_t,recall_t,fscore_t = get_precision(pred_t,l_t),get_recall(pred_t,l_t),get_fscore(pred_t,l_t)
        if fscore_t > fscore_max_t:
            fscore_max_t = fscore_t
            precision_t_max = precision_t
            recall_t_max = recall_t
            threshold_t = i
#         if fscore_d > fscore_max_d:
#             fscore_max_d = fscore_d
#             precision_d_max = precision_d
#             recall_d_max = recall_d
#             threshold_d = i
#         print("Threshold :%d ,precision in training: %f ,recall in t: %f,fcore in t: %f"% (i,precision_t,recall_t,fscore_t))
#         print("Threshold :%d ,precision in dev: %f ,recall in dev: %f,fcore in dev: %f"% (i,precision_d,recall_d,fscore_d))
    pred_d = mark(w_d,threshold_t)
    precision_d,recall_d,fscore_d = get_precision(pred_d,l_d),get_recall(pred_d,l_d),get_fscore(pred_d,l_d)
    print("Best performance in training, threshold : %d, precision %f, recall %f, fscore %f "%(threshold_t,precision_t_max,recall_t_max,fscore_max_t))
    print("Best performance in development, threshold : %d, precision %f, recall %f, fscore %f "%(threshold_t,precision_d,recall_d,fscore_d))
    training_performance = [precision_t, recall_t, fscore_t]
    development_performance = [precision_d, recall_d, fscore_d]
    return training_performance, development_performance    

In [56]:
training_file = "data/complex_words_training.txt"
development_file = "data/complex_words_development.txt"
print(word_length_threshold(training_file,development_file))

Best performance in training, threshold : 7, precision 0.598588, recall 0.851623, fscore 0.703030 
Best performance in development, threshold : 7, precision 0.605351, recall 0.866029, fscore 0.712598 
([0.7399150743099787, 0.5386398763523956, 0.6234347048300537], [0.6053511705685619, 0.8660287081339713, 0.7125984251968505])


In [57]:
## load Google Ngram counts
def load_ngram_counts(ngram_counts_file):
    counts = defaultdict(int)
    with gzip.open(ngram_counts_file,'rt') as f:
        for line in f:
            token,count = line.strip().split('\t')
            if token[0].islower():
                counts[token] = int(count)
    return counts
    

In [80]:
def frequency_threshold_feature(words, threshold, counts):
    result = []
    for i in range(len(words)):
        wordFreq = counts.get(words[i].lower())
        if wordFreq is None:
            result.append(1)
        elif wordFreq <= threshold:
            result.append(1)
        elif wordFreq > threshold:
            result.append(0)
    return  result

In [83]:
def word_frequency_threshold(training_file, development_file, counts):
    
    twords, tlabels = load_file(training_file)
    dwords, dlabels = load_file(development_file)
    max_in_t = find_max_frequency(counts, twords)
    max_in_d = find_max_frequency(counts, dwords)
    min_in_t = find_min_frequency(counts, twords)
    fscore_max = 0
    freq = 0
    for i in range(min_in_t, max_in_t + 1):
        preds = frequency_threshold_feature(twords,i, counts)
        fscore = get_fscore(preds, tlabels)
        fscore_max,freq = (fscore,i) if fscore > fscore_max else (fscore_max,freq)
    
    print("best freqency %d"% freq)
    dpred = frequency_threshold_feature(dwords, freq, counts)
    tpred = frequency_threshold_feature(twords, freq,counts)
    
    dprecision,drecall,dfscore = get_precision(dpred,dlabels),get_recall(dpred,dlabels),get_fscore(dpred,dlabels)
    tprecision,trecall,tfscore = get_precision(tpred,tlabels),get_recall(tpred,tlabels),get_fscore(tpred,tlabels)
    
    training_performance = [tprecision, trecall, tfscore]
    development_performance = [dprecision, drecall, dfscore]
    return training_performance, development_performance

In [65]:
def find_max_frequency(counts, words):
    max_freq = 0
    for word in words:
        if word in counts:
            freq = counts.get(word)
            max_freq = freq if freq > max_freq else max_freq
    return max_freq

In [69]:
def find_min_frequency(counts, words):
    min_freq = 1000
    for word in words:
        if word in counts:
            freq = counts.get(word)
            min_freq = freq if freq < min_freq else min_freq
    return min_freq

In [84]:
ngram_counts_file = "ngram_counts.txt.gz"
counts = load_ngram_counts(ngram_counts_file)
print(word_frequency_threshold(training_file,development_file,counts))

(0.06624383368569414, 0)


In [79]:
for key,val in counts.items():
    if val == 1120679362:
        print(key)

time
